In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import matplotlib.pyplot as plt


In [2]:
all_X = pandas.read_csv('./train_sensor_data_submit.csv',encoding = 'unicode_escape')

train_y = pandas.read_csv('./train_actuator_data_v1.csv',encoding = 'unicode_escape',dtype=str)
train_y.drop('ID',inplace=True, axis=1)

val_y = pandas.read_csv('./val_actuator_data_v1.csv',encoding = 'unicode_escape',dtype=str)
val_y.drop('ID',inplace=True,axis=1)

le = preprocessing.LabelEncoder()
le = le.fit(train_y['device3'].values)

train_y['device3'] = le.transform(train_y['device3'].values)
val_y['device3'] = le.transform(val_y['device3'].values)

all_y = pandas.concat([train_y, val_y], ignore_index=True, sort=False)

test_X = pandas.read_csv('./test_sensor_data_submit.csv',encoding = 'unicode_escape')


all_X

,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,CO2,...,rdiffBC_HUMI,rdiffvBC_HUMI,rdiffC_HUMI,diffCA_HUMI,diffvCA_HUMI,aggCA_HUMI,rdiffCA_HUMI,rdiffvCA_HUMI,aggABC_HUMI,rdiff_HUMI
0,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6338.000000,...,0.018397,1.400002,-0.014455,0.014667,1.099998,150.0,-0.014455,-1.099998,230.0,0.002464
1,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,3000.000000,...,-0.103139,-9.199997,-0.103139,0.129114,10.199997,158.0,-0.114350,-10.199997,238.0,-0.041457
2,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,2138.000000,...,-0.140709,-13.099998,-0.140709,0.178481,14.099998,158.0,-0.151450,-14.099998,238.0,-0.057843
3,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,6488.200000,...,0.061321,5.199997,0.061321,-0.057778,-5.199997,180.0,0.061321,5.199997,270.0,0.019259
4,6000.0,14.0,90.0,7,0,6000,14.5,86.0,11,6094.000000,...,-0.106957,-10.300003,-0.106957,0.094318,8.300003,176.0,-0.086189,-8.300003,262.0,-0.034224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312604,3000.0,20.0,90.0,7,0,3000,21.0,90.0,11,1152.000000,...,-0.118236,-11.800003,-0.138277,0.134091,11.800003,176.0,-0.118236,-11.800003,266.0,-0.041855
312605,3000.0,18.0,90.0,6,0,3000,18.0,90.0,11,6522.466797,...,-0.052814,-5.018333,-0.052814,0.055759,5.018333,180.0,-0.052814,-5.018333,270.0,-0.018586
312606,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6274.000000,...,-0.006410,-0.500000,-0.038462,0.040000,3.000000,150.0,-0.038462,-3.000000,230.0,-0.005797
312607,3000.0,13.0,80.0,6,0,3000,13.5,80.0,11,6520.000000,...,-0.182547,-17.864998,-0.182547,0.223312,17.864998,160.0,-0.182547,-17.864998,240.0,-0.074437


In [3]:
## one model for one device*+-*

all_Xcopy = all_X.copy()
test_Xcopy = test_X.copy()
cols = np.array([1,6,9,7,4,5,8,10,2,3])
preds_test = {}
train_X_dev = all_Xcopy

for d in cols:
    dev = 'device'+str(d)
    print(dev)
    train_y_dev = all_y[dev].values.astype('int')
    rf= RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                max_depth=25,# 20~30 
                max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=2,# 2,3,4
                min_samples_split=2,
                min_weight_fraction_leaf=0, n_estimators=200, n_jobs=8,
                oob_score=False, random_state=912, verbose=0,
                warm_start=False)
    # rf = AdaBoostClassifier(base_estimator=rf,n_estimators=300,random_state=912)
    rf.fit(train_X_dev.values,train_y_dev)

    pred_X=rf.predict(train_X_dev.values)
    train_X_dev[dev]=pred_X

    preds_test[dev] = rf.predict(test_Xcopy.values)#[test_Xcopy.columns[1:]])
    test_Xcopy[dev]=preds_test[dev]


device1
device6
device9
device7
device4
device5
device8
device10
device2
device3


In [5]:
test_y = pandas.DataFrame()
for dev in train_y.columns:
    test_y[dev]=preds_test[dev]
test_y.index.name = 'ID'
test_y['device3'] = le.inverse_transform(test_y['device3'])
print(test_y['device3'].unique())
test_y.to_csv('./submit_answer.csv')#,index=False)
test_y

['0' '1' '0.316666663' '0.966666639' '0.783333361' '0.616666675'
 '0.333333343' '0.13333334' '0.416666657' '0.233333334' '0.400000006'
 '0.300000012' '0.150000006' '0.200000003' '0.899999976' '0.583333313'
 '0.050000001' '0.25']


,device1,device2,device3,device4,device5,device6,device7,device8,device9,device10
ID,,,,,,,,,,
0,3,0,0,0,0,1,0,0,0,1
1,3,0,0,0,0,1,1,0,3,1
2,3,0,0,1,0,1,1,0,0,0
3,3,0,0,1,0,1,1,0,0,0
4,3,0,0,0,0,1,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...
78148,2,0,0,0,0,1,0,0,3,0
78149,3,0,0,0,0,1,1,0,3,1
78150,2,0,0,0,0,1,0,0,3,1
